In [7]:
# --- 1. Импорты и общие настройки ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet  # pip install prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")
# plt.style.use("seaborn-whitegrid")



In [8]:
# --- 2. Загрузка данных ---
# df должен содержать столбцы: "Регион", "Период" (YYYY-MM), и целевой столбец с показателем.
df = pd.read_excel("Датасет по верблюдам.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df.sample(10)



,Регион,Период,Верблюды
1246,КАРАГАНДИНСКАЯ ОБЛАСТЬ,2022-04-01,2.40
850,ГШЫМКЕНТ,2019-04-01,0.00
5,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-06-01,0.00
167,АКТЮБИНСКАЯ ОБЛАСТЬ,2018-12-01,527.66
560,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2021-09-01,7.60
625,ГАЛМАТЫ,2017-02-01,0.00
1351,КОСТАНАЙСКАЯ ОБЛАСТЬ,2021-01-01,0.00
1188,КАРАГАНДИНСКАЯ ОБЛАСТЬ,2017-06-01,14.70
2034,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2023-08-01,181.83
1452,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2019-06-01,2.70


In [9]:
regions = df['Регион'].unique()
target   = "Верблюды"
horizon  = 3
epsilon = 1e-6

In [10]:
first_test = pd.to_datetime("2024-01-01")
last_possible = df["Период"].max() - pd.DateOffset(months=horizon-1)
test_starts = pd.date_range(first_test, last_possible, freq="MS")

## Holw-Winter's (log)

In [11]:
results_hw = []
for region in regions:
    ts = (df[df["Регион"] == region]
          .set_index("Период")[target]
          .dropna()
          .sort_index())
    if len(ts) < 24:
        print(f"{region}: всего {len(ts)} мес. — сезонный Holt-Winter's невозможен.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        # формируем train / test
        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]

        # пропускаем, если недостаточно данных или неполный test
        if len(train) < 24 or len(test) < horizon:
            continue

        # обучаем модель
        train_log = np.log1p(train)

        hw_log = ExponentialSmoothing(
            train_log,
            seasonal="add",
            seasonal_periods=12
        ).fit(optimized=True)

        # прогноз и метрики
        fc_log = hw_log.forecast(horizon)
        fc = np.expm1(fc_log) 
        # fc   = model.forecast(horizon)
        rmse = np.sqrt(mean_squared_error(test, fc))
        mae  = mean_absolute_error(test, fc)
        mape = (np.abs((test - fc) / test).mean()) * 100

        results_hw.append({
            "Регион":      region,
            "Test start":  test_start.strftime("%Y-%m"),
            "Test end":    test_end.strftime("%Y-%m"),
            "Forecast":    [x.round(2) for x in list(fc.values)],
            "Actual":      [y.round(2) for y in list(test.values)],
            "RMSE":        rmse,
            "MAE":         mae,
            "MAPE_%":      mape
        })

# 4) Усреднение по всем скользящим окнам для каждого региона
res_hw = pd.DataFrame(results_hw)
print("Результаты прогнозов HW на 3 месяца:")

display(res_hw)

final_hw = (
    res_hw
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Holt–Winter's по регионам (rolling-3):")
display(final_hw)

Результаты прогнозов HW на 3 месяца:


,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[-0.11, -0.05, 0.04]","[0.0, 0.0, 0.0]",0.07,0.06,inf
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[-0.05, 0.04, -0.04]","[0.0, 0.0, 0.0]",0.04,0.04,inf
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[0.04, -0.04, -0.08]","[0.0, 0.0, 0.0]",0.06,0.06,inf
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[-0.04, -0.08, 0.3]","[0.0, 0.0, 0.0]",0.18,0.14,inf
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[-0.08, 0.31, -0.05]","[0.0, 0.0, 0.0]",0.19,0.15,inf
...,...,...,...,...,...,...,...,...
180,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[98.32, 41.95, 196.96]","[103.21, 38.88, 142.22]",31.78,20.90,17.04
181,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[42.41, 199.07, 97.95]","[38.88, 142.22, 138.52]",40.37,33.65,26.12
182,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[195.37, 96.14, 126.25]","[142.22, 138.52, 135.1]",39.58,34.79,24.84
183,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[89.35, 117.46, 198.48]","[138.52, 135.1, 262.18]",47.57,43.50,24.28


Средние метрики Holt–Winter's по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0.12,0.11,NaN
1,АКТЮБИНСКАЯ ОБЛАСТЬ,4.54,3.83,3.20
2,АЛМАТИНСКАЯ ОБЛАСТЬ,10.67,9.36,69.73
3,АТЫРАУСКАЯ ОБЛАСТЬ,62.88,46.21,11.05
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.66,0.54,NaN
5,ГАЛМАТЫ,0.00,0.00,NaN
6,ГАСТАНА,0.06,0.03,NaN
7,ГШЫМКЕНТ,0.00,0.00,NaN
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2.98,2.40,9.04
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2.25,1.69,13.07


In [12]:
res_hw[res_hw['Регион'] == 'ПАВЛОДАРСКАЯ ОБЛАСТЬ']

,Регион,Test start,Test end,Forecast,Actual,RMSE,MAE,MAPE_%
155,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-01,2024-03,"[0.17, 0.06, 0.08]","[0.0, 0.42, 0.0]",0.23,0.20,inf
156,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-02,2024-04,"[0.06, 0.08, 0.39]","[0.42, 0.0, 0.0]",0.31,0.28,inf
157,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-03,2024-05,"[0.08, 0.39, 0.08]","[0.0, 0.0, 0.0]",0.23,0.18,inf
158,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-04,2024-06,"[0.39, 0.08, 0.09]","[0.0, 0.0, 0.0]",0.24,0.19,inf
159,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-05,2024-07,"[0.08, 0.09, 0.0]","[0.0, 0.0, 0.0]",0.07,0.06,inf
160,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-06,2024-08,"[0.09, -0.0, 0.07]","[0.0, 0.0, 0.4]",0.20,0.14,inf
161,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-07,2024-09,"[-0.0, 0.07, 0.13]","[0.0, 0.4, 0.0]",0.21,0.15,inf
162,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-08,2024-10,"[0.07, 0.13, -0.0]","[0.4, 0.0, 0.0]",0.21,0.15,inf
163,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-09,2024-11,"[0.13, -0.0, 0.15]","[0.0, 0.0, 0.0]",0.11,0.09,inf
164,ПАВЛОДАРСКАЯ ОБЛАСТЬ,2024-10,2024-12,"[-0.0, 0.15, 0.38]","[0.0, 0.0, 0.0]",0.24,0.18,inf


## SARIMA

In [15]:
results_sarima = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    ts = ts + epsilon
    ts_log = np.log(ts)

    if len(ts_log) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. для авто-ARIMA, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train_log = ts_log[ts_log.index < test_start]
        test_log  = ts_log[(ts_log.index >= test_start) & (ts_log.index <= test_end)]
        if len(train_log) < 12 + horizon or len(test_log) < horizon:
            continue

        # автоподбор на лог-данных
        use_seasonal = len(train_log) >= 2 * 12

        sarima_log = auto_arima(
            train_log,
            seasonal=use_seasonal,
            m=12 if use_seasonal else 1,
            D=1 if use_seasonal else 0,      # фиксируем порядок сезонной разности
            seasonal_test=None,               # пропустить nsdiffs
            boxcox=True,
            stepwise=True,
            suppress_warnings=True,
            error_action="ignore"
        )
      
        # прогноз в лог-шкале
        fc_log = sarima_log.predict(n_periods=horizon, return_conf_int=False)

        # возвращаем прогноз в исходные единицы
        fc = np.exp(fc_log) - epsilon
        actual = np.exp(test_log.values) - epsilon  # но exp(log(x)) == x

        # метрики на исходном уровне
        rmse = np.sqrt(mean_squared_error(actual, fc))
        mae  = mean_absolute_error(actual, fc)
        mape = (np.abs((actual - fc) / actual).mean()) * 100

        results_sarima.append({
            "Регион":         region,
            "Test start":     test_start.strftime("%Y-%m"),
            "Test end":       test_end.strftime("%Y-%m"),
            "order":          sarima_log.order,
            "seasonal_order": sarima_log.seasonal_order,
            "RMSE":           round(rmse,2),
            "MAE":            round(mae,2),
            "MAPE_%":         round(mape,2),
            "Forecast":       [round(x,2) for x in fc],
            "Actual":         [round(y,2) for y in actual]
        })
# формируем DataFrame с результатами
res_sarima = pd.DataFrame(results_sarima)
print("Результаты прогнозов SARIMA на 3 месяца:")

display(res_sarima)

final_sarima = (
    res_sarima
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики SARIMA по регионам (rolling-3):")
display(final_sarima)

Результаты прогнозов SARIMA на 3 месяца:


,Регион,Test start,Test end,order,seasonal_order,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,"(0, 0, 1)","(0, 1, 1, 12)",0.00,0.00,407346345275563136.00,"[-0.0, 0.0, -0.0]","[0.0, 0.0, 0.0]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,"(0, 0, 1)","(0, 1, 1, 12)",0.00,0.00,297018887419731200.00,"[0.0, -0.0, -0.0]","[0.0, 0.0, 0.0]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,"(0, 0, 1)","(0, 1, 1, 12)",0.00,0.00,125196112894837824.00,"[-0.0, -0.0, -0.0]","[0.0, 0.0, 0.0]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,"(0, 0, 1)","(0, 1, 1, 12)",0.00,0.00,1102837260518193408.00,"[-0.0, -0.0, 0.0]","[0.0, 0.0, 0.0]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,"(0, 0, 1)","(0, 1, 1, 12)",0.00,0.00,1083282317046571392.00,"[-0.0, 0.0, -0.0]","[0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,"(0, 1, 2)","(0, 1, 1, 12)",15.65,13.50,12.62,"[84.08, 36.58, 161.28]","[103.21, 38.88, 142.22]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,"(0, 1, 2)","(0, 1, 1, 12)",23.78,19.90,15.11,"[37.36, 172.22, 110.35]","[38.88, 142.22, 138.52]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,"(0, 1, 2)","(0, 1, 1, 12)",26.61,26.24,18.87,"[173.63, 111.81, 114.51]","[142.22, 138.52, 135.1]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,"(0, 1, 1)","(0, 1, 0, 12)",48.06,45.76,25.70,"[102.01, 100.79, 195.71]","[138.52, 135.1, 262.18]"


Средние метрики SARIMA по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,449141870281272064.00
1,АКТЮБИНСКАЯ ОБЛАСТЬ,6.45,5.56,4.90
2,АЛМАТИНСКАЯ ОБЛАСТЬ,10.08,8.78,64.91
3,АТЫРАУСКАЯ ОБЛАСТЬ,82.15,63.60,14.69
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.29,0.18,1584955661707730223104.00
5,ГАЛМАТЫ,1.00,1.00,236118088025158109364224.00
6,ГАСТАНА,0.00,0.00,13.33
7,ГШЫМКЕНТ,1.00,1.00,236118088025158109364224.00
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2.76,2.22,8.92
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,3.61,2.85,24.70


## Facebook Prophet

In [13]:
results_prophet = []

for region in regions:
    ts = (
        df[df["Регион"] == region]
        .set_index("Период")[target]
        .dropna()
        .sort_index()
    )
    if len(ts) < 12 + horizon:
        print(f"{region}: менее {12+horizon} мес. данных, пропускаем.")
        continue

    for test_start in test_starts:
        test_end = test_start + pd.DateOffset(months=horizon) - pd.DateOffset(days=1)

        train = ts[ts.index < test_start]
        test  = ts[(ts.index >= test_start) & (ts.index <= test_end)]
        if len(train) < 12 + horizon or len(test) < horizon:
            continue

        # Подготовка данных для Prophet
#         df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
# # подготовка для одного региона
        df_prophet = train.reset_index().rename(columns={"Период":"ds", target:"y"})
        df_prophet["y"] = np.log(df_prophet["y"] + epsilon)

        m = Prophet()
        m.fit(df_prophet)

        future = m.make_future_dataframe(periods=horizon, freq="MS")
        forecast = m.predict(future)

        # берем только прогнозные точки
        yhat_log = forecast["yhat"].values[-horizon:]
        fc = np.exp(yhat_log) - epsilon

        # m = Prophet()
        # m.fit(df_prophet)

        # # Создаем DataFrame будущих дат и делаем прогноз
        # # future = m.make_future_dataframe(periods=horizon, freq="MS")
        # # forecast = m.predict(future)

        # # Отбираем только наши горизонты
        # fc = forecast.set_index("ds")["yhat"].loc[test.index].values
        actual = test.values

        # Расчет метрик
        rmse  = np.sqrt(mean_squared_error(actual, fc))
        mae   = mean_absolute_error(actual, fc)
        mape  = (np.abs((actual - fc) / actual).mean()) * 100

        results_prophet.append({
            "Регион":     region,
            "Test start": test_start.strftime("%Y-%m"),
            "Test end":   test_end.strftime("%Y-%m"),
            "RMSE":       round(rmse, 2),
            "MAE":        round(mae, 2),
            "MAPE_%":     round(mape, 2),
            "Forecast":   [round(x, 2) for x in fc],
            "Actual":     [round(x, 2) for x in actual]
        })

# Собираем результаты в DataFrame
res_prophet = pd.DataFrame(results_prophet)
print("Результаты прогнозов Prophet на 3 месяца:")
display(res_prophet)

final_prophet = (
    res_prophet
    .groupby("Регион")[["RMSE","MAE","MAPE_%"]]
    .mean()
    .round(2)
    .reset_index()
)
print("Средние метрики Prophet по регионам (rolling-3):")
display(final_prophet)


19:27:58 - cmdstanpy - INFO - Chain [1] start processing
19:27:58 - cmdstanpy - INFO - Chain [1] done processing
19:27:58 - cmdstanpy - INFO - Chain [1] start processing
19:27:58 - cmdstanpy - INFO - Chain [1] done processing
19:27:58 - cmdstanpy - INFO - Chain [1] start processing
19:27:58 - cmdstanpy - INFO - Chain [1] done processing
19:27:59 - cmdstanpy - INFO - Chain [1] start processing
19:27:59 - cmdstanpy - INFO - Chain [1] done processing
19:27:59 - cmdstanpy - INFO - Chain [1] start processing
19:27:59 - cmdstanpy - INFO - Chain [1] done processing
19:27:59 - cmdstanpy - INFO - Chain [1] start processing
19:27:59 - cmdstanpy - INFO - Chain [1] done processing
19:27:59 - cmdstanpy - INFO - Chain [1] start processing
19:27:59 - cmdstanpy - INFO - Chain [1] done processing
19:27:59 - cmdstanpy - INFO - Chain [1] start processing
19:27:59 - cmdstanpy - INFO - Chain [1] done processing
19:28:00 - cmdstanpy - INFO - Chain [1] start processing
19:28:00 - cmdstanpy - INFO - Chain [1]

Результаты прогнозов Prophet на 3 месяца:


,Регион,Test start,Test end,RMSE,MAE,MAPE_%,Forecast,Actual
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-01,2024-03,0.00,0.00,inf,"[-0.0, 0.0, 0.0]","[0.0, 0.0, 0.0]"
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-02,2024-04,0.00,0.00,inf,"[-0.0, 0.0, -0.0]","[0.0, 0.0, 0.0]"
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-03,2024-05,0.00,0.00,inf,"[0.0, -0.0, -0.0]","[0.0, 0.0, 0.0]"
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-04,2024-06,0.00,0.00,inf,"[-0.0, -0.0, 0.0]","[0.0, 0.0, 0.0]"
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2024-05,2024-07,0.00,0.00,inf,"[-0.0, 0.0, -0.0]","[0.0, 0.0, 0.0]"
...,...,...,...,...,...,...,...,...
195,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-06,2024-08,25.05,22.14,21.92,"[78.56, 45.57, 177.3]","[103.21, 38.88, 142.22]"
196,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-07,2024-09,40.65,35.33,30.31,"[47.09, 183.71, 82.23]","[38.88, 142.22, 138.52]"
197,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,2024-10,41.49,38.34,27.55,"[181.21, 81.08, 116.5]","[142.22, 138.52, 135.1]"
198,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,2024-11,56.27,51.68,29.00,"[79.29, 113.66, 187.8]","[138.52, 135.1, 262.18]"


Средние метрики Prophet по регионам (rolling-3):


,Регион,RMSE,MAE,MAPE_%
0,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,NaN
1,АКТЮБИНСКАЯ ОБЛАСТЬ,13.02,10.63,9.01
2,АЛМАТИНСКАЯ ОБЛАСТЬ,54.35,40.27,328.09
3,АТЫРАУСКАЯ ОБЛАСТЬ,135.06,110.55,24.34
4,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.29,0.17,NaN
5,ГАЛМАТЫ,0.00,0.00,NaN
6,ГАСТАНА,0.00,0.00,NaN
7,ГШЫМКЕНТ,0.00,0.00,NaN
8,ЖАМБЫЛСКАЯ ОБЛАСТЬ,15.50,12.07,35.17
9,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,5.12,4.23,50.79


In [16]:
# Переименуем колонки с MAPE, чтобы было понятно, к какому методу относятся
hw = final_hw.rename(columns={"MAPE_%": "MAPE_HW"})
sar = final_sarima.rename(columns={"MAPE_%": "MAPE_SARIMA"})
pr  = final_prophet.rename(columns={"MAPE_%": "MAPE_Prophet"})

# Мёрджим по региону
summary = (
    hw[["Регион", "MAPE_HW"]]
    .merge(sar[["Регион", "MAPE_SARIMA"]], on="Регион")
    .merge(pr[["Регион", "MAPE_Prophet"]], on="Регион")
)

# Определяем для каждой строки, какой столбец MAPE минимален
# idxmin вернёт название столбца с минимальным значением
summary["Best_method"] = summary[["MAPE_HW","MAPE_SARIMA","MAPE_Prophet"]] \
                           .idxmin(axis=1) \
                           .str.replace("MAPE_","")  # убираем префикс для красоты

# Если нужно, можно сразу отсортировать
# summary = summary.sort_values("Best_method")

# допустим, у вас уже есть summary
summary = summary.round({
    "MAPE_HW": 2,
    "MAPE_SARIMA": 2,
    "MAPE_Prophet": 2
})

# Готово!
print(summary.to_string(index=False))


                        Регион  MAPE_HW                 MAPE_SARIMA  MAPE_Prophet Best_method
           АКМОЛИНСКАЯ ОБЛАСТЬ      NaN       449141870281272064.00           NaN      SARIMA
           АКТЮБИНСКАЯ ОБЛАСТЬ     3.20                        4.90          9.01          HW
           АЛМАТИНСКАЯ ОБЛАСТЬ    69.73                       64.91        328.09      SARIMA
            АТЫРАУСКАЯ ОБЛАСТЬ    11.05                       14.69         24.34          HW
ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ      NaN   1584955661707730223104.00           NaN      SARIMA
                       ГАЛМАТЫ      NaN 236118088025158109364224.00           NaN      SARIMA
                       ГАСТАНА      NaN                       13.33           NaN      SARIMA
                      ГШЫМКЕНТ      NaN 236118088025158109364224.00           NaN      SARIMA
            ЖАМБЫЛСКАЯ ОБЛАСТЬ     9.04                        8.92         35.17      SARIMA
 ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ    13.07                     